In [8]:
import pandas as pd

df = pd.read_csv("df_features.csv")
df.head()


,BuildingType,PrimaryPropertyType,ZipCode,CouncilDistrictCode,Neighborhood,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,...,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearsENERGYSTARCertified,ENERGYSTARScore,SiteEUIWN(kBtu/sf),SteamUse(kBtu),Agedubatiment,UsageCount,ParkingRatio,PrimaryUseRatio
0,NonResidential,Hotel,98101.0,7,DOWNTOWN,47.61220,-122.33799,1927,1.0,12,...,Unknown,12371.421298,Unknown,60.0,84.300003,2003882.00,98,1,0.000000,1.000000
1,NonResidential,Hotel,98101.0,7,DOWNTOWN,47.61317,-122.33393,1996,1.0,11,...,Restaurant,4622.000000,Unknown,61.0,97.900002,0.00,29,3,0.145453,0.809918
2,NonResidential,Hotel,98101.0,7,DOWNTOWN,47.61393,-122.33810,1969,1.0,41,...,Unknown,12371.421298,Unknown,43.0,97.699997,21566554.00,56,1,0.205748,0.791220
3,NonResidential,Hotel,98101.0,7,DOWNTOWN,47.61412,-122.33664,1926,1.0,10,...,Unknown,12371.421298,Unknown,56.0,113.300003,2214446.25,99,1,0.000000,1.000000
4,NonResidential,Hotel,98121.0,7,DOWNTOWN,47.61375,-122.34047,1980,1.0,18,...,Swimming Pool,0.000000,Unknown,75.0,118.699997,0.00,45,3,0.353115,0.703070


## 4.1 – Définition de la variable cible et des features

In [9]:
TARGET = "SiteEUIWN(kBtu/sf)"

X = df.drop(columns=[TARGET])
y = df[TARGET]

print("X :", X.shape)
print("y :", y.shape)


X : (1521, 27)
y : (1521,)


## 4.2 – Train/Test Split (80% / 20%)

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

print("X_train :", X_train.shape)
print("X_test  :", X_test.shape)


X_train : (1216, 27)
X_test  : (305, 27)


## 4.3 – Préprocessing : pipeline (scaling + encodage)

Dans cette étape, on prépare les données pour les modèles de machine learning.
L’idée est de tout automatiser pour éviter les erreurs et surtout le data leakage.

On va donc créer un pipeline de prétraitement qui :

met à l’échelle les variables numériques avec StandardScaler
(pour que toutes soient sur la même échelle) ;

transforme les variables catégorielles avec OneHotEncoder
(pour convertir les catégories en colonnes numériques) ;

rassemble tout dans un ColumnTransformer.

Ce pipeline sera ajouté à chaque modèle pour que le même prétraitement soit appliqué sur le train et le test.


In [11]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Colonnes numériques / catégorielles repérées à partir de X
numeric_features = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_features = X.select_dtypes(include=["object"]).columns.tolist()

print("Numériques :", numeric_features)
print("Catégorielles :", categorical_features)

numeric_transformer = Pipeline(steps=[
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)


Numériques : ['ZipCode', 'CouncilDistrictCode', 'Latitude', 'Longitude', 'YearBuilt', 'NumberofBuildings', 'NumberofFloors', 'PropertyGFATotal', 'PropertyGFAParking', 'PropertyGFABuilding(s)', 'LargestPropertyUseTypeGFA', 'SecondLargestPropertyUseTypeGFA', 'ThirdLargestPropertyUseTypeGFA', 'ENERGYSTARScore', 'SteamUse(kBtu)', 'Agedubatiment', 'UsageCount', 'ParkingRatio', 'PrimaryUseRatio']
Catégorielles : ['BuildingType', 'PrimaryPropertyType', 'Neighborhood', 'ListOfAllPropertyUseTypes', 'LargestPropertyUseType', 'SecondLargestPropertyUseType', 'ThirdLargestPropertyUseType', 'YearsENERGYSTARCertified']


## 4.4 – Modèle de base : régression linéaire

On construit un premier modèle simple de régression linéaire.
Le pipeline applique d’abord le préprocessing (scaling + one-hot), puis entraîne le modèle sur `X_train`, `y_train`.


In [12]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

# Prédictions
y_pred_lr = model_lr.predict(X_test)

# Métriques
r2_lr = r2_score(y_test, y_pred_lr)
mae_lr = mean_absolute_error(y_test, y_pred_lr)

mse_lr = mean_squared_error(y_test, y_pred_lr)   # MSE
rmse_lr = np.sqrt(mse_lr)                        # RMSE = racine de MSE

mape_lr = np.mean(np.abs((y_test - y_pred_lr) / y_test)) * 100

print("Régression linéaire")
print("R²     :", r2_lr)
print("MAE    :", mae_lr)
print("RMSE   :", rmse_lr)
print("MAPE % :", mape_lr)


Régression linéaire
R²     : 0.34632532177608055
MAE    : 39.324668649239946
RMSE   : 65.83598375247554
MAPE % : inf


## 4.5 – Modèle non linéaire : RandomForest

On teste un deuxième modèle plus flexible (RandomForest) avec le même pipeline de prétraitement, puis on compare les métriques.


In [13]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

# Prédictions
y_pred_rf = model_rf.predict(X_test)

# Métriques
r2_rf = r2_score(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)

mse_rf = mean_squared_error(y_test, y_pred_rf)  # MSE
rmse_rf = np.sqrt(mse_rf)                       # RMSE = racine du MSE

mape_rf = np.mean(np.abs((y_test - y_pred_rf) / y_test)) * 100

print("RandomForest")
print("R²     :", r2_rf)
print("MAE    :", mae_rf)
print("RMSE   :", rmse_rf)
print("MAPE % :", mape_rf)


RandomForest
R²     : 0.44982419212099434
MAE    : 30.823460974356962
RMSE   : 60.399486087489215
MAPE % : inf


## 4.6 – Modèle non linéaire : Gradient Boosting

On teste un troisième modèle (GradientBoostingRegressor) avec le même pipeline de prétraitement, puis on compare ses performances aux modèles précédents.


In [14]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

# Pipeline Gradient Boosting
model_gb = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", GradientBoostingRegressor(random_state=42))
])

# Entraînement
model_gb.fit(X_train, y_train)

# Prédictions
y_pred_gb = model_gb.predict(X_test)

# Métriques
r2_gb = r2_score(y_test, y_pred_gb)
mae_gb = mean_absolute_error(y_test, y_pred_gb)

mse_gb = mean_squared_error(y_test, y_pred_gb)   # MSE
rmse_gb = np.sqrt(mse_gb)                        # RMSE

mape_gb = np.mean(np.abs((y_test - y_pred_gb) / y_test)) * 100

print("Gradient Boosting")
print("R²     :", r2_gb)
print("MAE    :", mae_gb)
print("RMSE   :", rmse_gb)
print("MAPE % :", mape_gb)


Gradient Boosting
R²     : 0.3822866589500076
MAE    : 32.76347583415546
RMSE   : 63.99941276088994
MAPE % : inf


### 4.7 - Comparaison des modeles

In [17]:
compare = pd.DataFrame({
    "Modèle": ["Régression linéaire", "RandomForest", "GradientBoosting"],
    "R²": [r2_lr, r2_rf, r2_gb],
    "MAE": [mae_lr, mae_rf, mae_gb],
    "RMSE": [rmse_lr, rmse_rf, rmse_gb]
})

compare


,Modèle,R²,MAE,RMSE
0,Régression linéaire,0.346325,39.324669,65.835984
1,RandomForest,0.449824,30.823461,60.399486
2,GradientBoosting,0.382287,32.763476,63.999413


### 4.8 Validation croisée

In [19]:
# Validation croisée – Régression linéaire

from sklearn.model_selection import cross_validate

# Validation croisée — Régression linéaire
cv_lr = cross_validate(
    model_lr,
    X_train, y_train,
    cv=5,
    scoring=["r2", "neg_mean_absolute_error", "neg_root_mean_squared_error"],
    n_jobs=-1
)

print("Régression linéaire (CV)")
print("R²     :", cv_lr["test_r2"].mean())
print("MAE    :", -cv_lr["test_neg_mean_absolute_error"].mean())
print("RMSE   :", -cv_lr["test_neg_root_mean_squared_error"].mean())


Régression linéaire (CV)
R²     : 0.28818718551560407
MAE    : 38.76761227455127
RMSE   : 62.28102325618073


In [20]:
# Validation croisée – RandomForest

from sklearn.model_selection import cross_validate


cv_rf = cross_validate(
    model_rf,
    X_train, y_train,
    cv=5,
    scoring=["r2", "neg_mean_absolute_error", "neg_root_mean_squared_error"],
    n_jobs=-1
)

print("RandomForest (CV)")
print("R²      :", cv_rf["test_r2"].mean())
print("MAE     :", -cv_rf["test_neg_mean_absolute_error"].mean())
print("RMSE    :", -cv_rf["test_neg_root_mean_squared_error"].mean())


RandomForest (CV)
R²      : 0.4965406703151703
MAE     : 30.744864608975075
RMSE    : 52.5671626001621


In [21]:
# Validation croisée – GradientBoosting

from sklearn.model_selection import cross_validate

cv_gb = cross_validate(
    model_gb,
    X_train, y_train,
    cv=5,
    scoring=["r2", "neg_mean_absolute_error", "neg_root_mean_squared_error"],
    n_jobs=-1
)

print("Gradient Boosting (CV)")
print("R²      :", cv_gb["test_r2"].mean())
print("MAE     :", -cv_gb["test_neg_mean_absolute_error"].mean())
print("RMSE    :", -cv_gb["test_neg_root_mean_squared_error"].mean())


Gradient Boosting (CV)
R²      : 0.49004241238743107
MAE     : 31.46414722689296
RMSE    : 52.67469503032921


In [ ]:
# Tableau comparatif final (cross-validation)

compare_cv = pd.DataFrame({
    "Modèle": ["Régression linéaire", "RandomForest", "GradientBoosting"],
    "R² (CV)": [
        cv_lr["test_r2"].mean(),
        cv_rf["test_r2"].mean(),
        cv_gb["test_r2"].mean()
    ],
    "MAE (CV)": [
        -cv_lr["test_neg_mean_absolute_error"].mean(),
        -cv_rf["test_neg_mean_absolute_error"].mean(),
        -cv_gb["test_neg_mean_absolute_error"].mean()
    ],
    "RMSE (CV)": [
        -cv_lr["test_neg_root_mean_squared_error"].mean(),
        -cv_rf["test_neg_root_mean_squared_error"].mean(),
        -cv_gb["test_neg_root_mean_squared_error"].mean()
    ]
})

compare_cv


,Modèle,R² (CV),MAE (CV),RMSE (CV)
0,Régression linéaire,0.288187,38.767612,62.281023
1,RandomForest,0.496541,30.744865,52.567163
2,GradientBoosting,0.490042,31.464147,52.674695


La validation croisée confirme les tendances observées avec le train/test split.

Le RandomForest reste le meilleur modèle, avec le R² moyen le plus élevé et les erreurs (MAE, RMSE) les plus faibles.

La régression linéaire a les performances les plus faibles, ce qui est attendu : les relations du dataset ne sont pas linéaires.

Le Gradient Boosting donne des résultats intermédiaires.

Les variations entre les folds sont raisonnables, ce qui montre que les modèles ne sont pas instables.